<p><img alt="Colaboratory logo" height="65px" src="https://upload.wikimedia.org/wikipedia/en/thumb/b/b1/Davivienda_logo.svg/1200px-Davivienda_logo.svg.png" align="left" hspace="10px" width="20%" vspace="15px"></p>

<h1 align="center"> Prueba Técnica Profesional III Departamento de Datos no Estructurados  </h1>


En el Departamento de Datos No Estructurados buscamos personas con excelentes capacidades técnicas a las que les guste explorar los últimos avances en IA para asumir retos de especial dificultad. Aunque la siguiente prueba busca, en primera instancia, corroborar su idoneidad técnica, para nosotros es muy importante que esta destreza venga acompañada de dos elementos adicionales:

1.   Creatividad para encarar desafíos técnicos en procesamiento de datos no estructurados.
2.   Habilidad para comunicar el trabajo hecho y sus resultados a un público general o experto.


---


### **Análisis de Tweets** 

Una de las actividades tipicas en el departamento es el analisis, mineria de texto y procesamiento de lenguaje natural para los diferentes procesos operativos o que tienen interacción con el Banco. Para este ejercicio se adjutará un base de datos llamada **davivienda_tweets.csv** o directamente puede descargarla empleando el siguiente comando.

```
!wget "https://raw.githubusercontent.com/yemoncada/davivienda_tweets/main/davivienda_tweets.csv" -P "/content"
```

Este archivo contiene un conjunto de **tweets extraidos de la red social** de Twitter que contienen la palabra **Davivienda** y con el cual quisieramos conocer cual es la interacción que tienen los diferentes usuarios de la red social con el Banco Davivienda. Todo esto a partir de una exploración general de lo que esta en los textos, cabe destacar que este notebook esta diseñado para que realice una exploración guiada y que por medio de esta vaya realizando sus propios descubrimientos e insights sobre la base de datos.

Por último y con la finalidad de obtener un analisis más detallado, realice un modelo no supervisado Latent Dirichlet Allocation (LDA) que permita conocer por topicos, cuales son los temas más hablados en Twitter asociados a la palabra Davivienda.

<p><img alt="Colaboratory logo" src="https://i.ibb.co/3zNV9vX/1-Gp-PEknl-KMQ09-Uk-Et-Y6-MZOw.png" align="left" hspace="10px" vspace="15px"></p>


***Nota***: *Considere que este ejercicio es hipotético y el banco no usará su trabajo más que para evaluar sus habilidades para el cargo*



---



## Mineria de Texto - Davivienda Tweets.

Con la finalidad de realizar el ejercicio de mineria y procesamiento de lenguaje natural puede utilizar cualquier libreria de python que facilite las siguientes tareas:

1. ***Analisis exploratorio*** y descriptivo de la base de datos davivienda_tweets.csv

2. ***Prepocesamiento del texto***:

    * Tokenizar: Separar el texto en párrafos, frases, etc..
    * Limpieza: Minúsculas, quito puntuación, remuevo palabras de 3 caracteres, etc. Utilice expresiones regulares para limpiar su texto.
    * Stopwords: estas palabras no tienen un significado por sí solas, sino que modifican o acompañan a otras.
    * Lematizar: cambio de tiempos verbales.
    * Estematizar o Stemmed: convertir palabras a sus raíces

3. ***WordClouds o Nubes de Palabras*** que permitan la visualización de datos  textuales significativos y conocer su relevancia en el texto.

4. ***Matriz termino-documento*** - Document-Term Matrix.

5. ***Histograma*** de palabras más importantes.

6. ***Latent Dirichlet Allocation*** - Modelo no supervisado de Tópicos.

7. ***Interpretación de Resultados y conclusiones generales.***


## 1. Análisis Exploratorio

En este a partado realice el análisis exploratorio y descriptivo de la base de datos davivienda_tweets.csv

In [ ]:
!wget "https://raw.githubusercontent.com/yemoncada/davivienda_tweets/main/davivienda_tweets.csv" -P "/content"

--2022-08-10 03:39:38--  https://raw.githubusercontent.com/yemoncada/davivienda_tweets/main/davivienda_tweets.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 677050 (661K) [text/plain]
Saving to: ‘/content/davivienda_tweets.csv.2’

davivienda_tweets.c 100%[===================>] 661.18K  --.-KB/s    in 0.05s   

2022-08-10 03:39:38 (13.2 MB/s) - ‘/content/davivienda_tweets.csv.2’ saved [677050/677050]



In [ ]:
import pandas as pd
import re
import string
import unidecode

import glob
 

import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
pal_vacias = nlp.Defaults.stop_words

## 2. Prepocesamiento de Texto

In [ ]:
def limpia_texto(data, columna = None):
        '''Función que realiza una limpieza del texto que esta contenido en una variable.
        Al finalizar generará una nueva columna llamada "texto_procesado"

        PARAMETROS:
        _________________________________________

        Columna: es la variable que contienen los textos que deben ser limpiados'''


        if columna is not None:

            textoProc = []

            for text in data.index:
                parrafo = data[columna][text].lower()  # minusculas
                regex = '[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
                parrafo = re.sub(regex, "", parrafo)  ##Eliminar signos de puntuación
                parrafo = re.sub("\d+", ' ', parrafo)  ##Eliminar numeros
                parrafo = re.sub("\\s+", ' ', parrafo)  ##Eliminar espacios en blanco múltiples
                parrafo = ' '.join([token for token in parrafo.split()])  ## unir lineas por espacio
                parrafo = unidecode.unidecode(parrafo)  ## Elimina caracteres especiales.
                textoProc.append(parrafo)
            data[columna] = textoProc

In [ ]:
def tokenizacion(data, columna = None):
        '''Función que permite realizar el proceso de tokenización del texto.
        Al finalizar, la función generará una columna llamada "token", que contendrá
        la lista de palabras separadas por coma.

        PARAMETROS
        ____________________________________________
        Columna: variable que contiene el texto a tokenizar.

        '''
        if columna is not None:
            tokens=[]
            for text in data.index:
                listaTokens = [token for token in data[columna][text].split() if token not in pal_vacias] 
                tokens.append(listaTokens)
            data['tokens'] = tokens

## Análisis No Supervisado: Modelo Clasificación Zero-Shot



In [ ]:
# Packages 

import json
import numpy as np
import pandas as pd
import tokenizer

from dateparser import parse
from tqdm.notebook import tqdm

import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, BertTokenizerFast

tqdm.pandas()
torch.cuda.empty_cache()

### Zero - Shot Topic Classification

In [ ]:
# Load spanish zero shot 
 
zero_shot_classifier = pipeline("zero-shot-classification", model = "Recognai/bert-base-spanish-wwm-cased-xnli")

### Apply models

In [ ]:
#  spa_eng_label_conversation, spa_eng, hate_detection, emotion_rating, emotion_classification, toxicity_classifier,

def sentiment_topic_analysis(text, zero_shot_classifier, zero_shot_labels): 

    if type(text) == str:

        topic = zero_shot_classifier(text, candidate_labels = zero_shot_labels, multi_label = False)
        
        topic_df = pd.DataFrame([topic['scores']], columns = topic['labels'])
        
#         final_df = pd.concat([sentiment_df, topic_df], axis = 1)
        
        topic_df['text'] = text
        
        return topic_df
        
    else:
        
        return None

In [ ]:
data = pd.read_csv('/content/davivienda_tweets.csv', index_col = 0).reset_index().dropna(subset = ['Embedded_text'])
#data = data.sample(n=5000)

In [ ]:
data['Timestamp'] = data['Timestamp'].progress_apply(lambda x: parse(x))

data = data.sort_values('Timestamp', ascending = False)

  0%|          | 0/1811 [00:00<?, ?it/s]

In [ ]:
dfs = []
for index in tqdm(np.arange(data.shape[0])): 

    try: 

        post_date = data.iloc[index]['Timestamp']
        text = data.iloc[index]['Embedded_text']

        df = sentiment_topic_analysis(text,
                                      #spa_eng_label_conversation,
                                      #spa_eng,
                                      #hate_detection,
                                      #emotion_rating,
                                      #emotion_classification,
                                      #toxicity,
                                      zero_shot_classifier,
                                      zero_shot_labels = ['excelente servicio',
                                                          'mal servicio'])

        df['PostDate'] = post_date

        dfs.append(df)

    except KeyboardInterrupt: 
        break

    except: 
        print(len(fails))

        pass

  0%|          | 0/1811 [00:00<?, ?it/s]

In [ ]:
post_sentiment = pd.concat(dfs).astype('float', errors = 'ignore')

post_sentiment['Max'] = post_sentiment[['excelente servicio',
                                                          'mal servicio']].idxmax(axis = 1)

In [ ]:
post_sentiment.to_csv('davi_comments_all_two.csv')

## Resultados

In [ ]:
import pandas
df = pandas.read_csv('/content/davi_comments_all_two.csv')
df

,Unnamed: 0,mal servicio,excelente servicio,text,PostDate,Max
0,0,0.636739,0.363261,En respuesta a \n@Xtresveces\nDaviplata no se ...,2021-12-22 23:53:50+00:00,mal servicio
1,0,0.885410,0.114590,En respuesta a \n@wilsonariasc\n @CLONYHD\n y ...,2021-12-22 23:32:32+00:00,mal servicio
2,0,0.440768,0.559232,En respuesta a \n@WILIRBOA\nBuenas tardes. De ...,2021-12-22 22:53:07+00:00,excelente servicio
3,0,0.154679,0.845321,"En respuesta a \n@gleniiaaa\nBuenas tardes, pa...",2021-12-22 22:26:00+00:00,excelente servicio
4,0,0.436653,0.563347,Respondiendo a \n@Bco_Occidente\n @Davivienda\...,2021-12-22 22:16:19+00:00,excelente servicio
...,...,...,...,...,...,...
1806,0,0.625693,0.374307,En respuesta a \n@KarenTralara\n y \n@metrodem...,2021-12-01 02:07:37+00:00,mal servicio
1807,0,0.477338,0.522662,"#daviplata ayuda por favor, llevo más de 3 mes...",2021-12-01 01:31:18+00:00,excelente servicio
1808,0,0.722058,0.277942,En respuesta a \n@oskar767\n y \n@johngongo\nL...,2021-12-01 00:38:14+00:00,mal servicio
1809,0,0.275838,0.724162,En respuesta a \n@WhitneyMeneses\n @Davivienda...,2021-12-01 00:04:36+00:00,excelente servicio


In [ ]:
df['Max'].iloc[0] # Primera fila

'mal servicio'

In [ ]:
df['text'].iloc[0] # Primera fila

'En respuesta a \n@Xtresveces\nDaviplata no se cae (al menos desde que la uso hace dos años nunca ha fallado), pero casi nadie la usa y hay pocos corresponsales de davivienda :('